# Dynamically pin and unpin from providers
## Benefit 
* Data Scientists can dynamically change pinning providers
* Data Scientists don't rely on one IPFS gateway, but can use many


## Importing libraries

In [1]:
#main protocol
from storage.ipfs import IPFS

#pinning providers
from storage.nftstorage import NFTStorage
from storage.pinataV1 import PinataV1


#helpers
from helpers.helper import read_file
import pandas as pd

In [2]:
#Load in credentials - Not an optimal solution and can investigate a better solution with something like Lit Protocol

nft_storage = NFTStorage()
nft_storage_creds = nft_storage.get_creds()
ipfs = IPFS()
pinata = PinataV1()
pinata_creds = pinata.get_creds()

In [6]:
#Upload to nft storage, this is called pinning or simplicity, NFTstorage actually facilitates the pinning deals under the hood inteh deals section
#the response returns a cid and what filecoin deals have been made to pin your file.

response, status_code = nft_storage.upload_file(nft_storage_creds,"dataset/ens_airdrop.json")

pd.DataFrame(response)

,ok,value
cid,True,bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rv...
created,True,2022-03-09T02:21:11.461+00:00
deals,True,"[{'status': 'active', 'lastChanged': '2022-03-..."
files,True,[]
name,True,Upload at 2022-05-17T18:04:01.593Z
pin,True,{'cid': 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4...
scope,True,audiocentive
size,True,42725339
type,True,form-data


In [7]:
#Pin to pinata for redundency
pinata.pin(pinata_creds,response["value"]["cid"],fn="ens_airdrop.json")

({'id': 'f9878627-7086-4018-864c-6bb26e5b553d',
  'ipfsHash': 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rvwkpjfr6tsj3ye',
  'status': 'prechecking',
  'name': 'ens_airdrop.json'},
 200)

In [8]:
#Check the status of the pinning job, pin policy will show what regions the file is available in
params = {"hashContains":"DESC","status":None,"prechecking":None,
          "searching":None,"retrieving":None,
          "expired":None,"over_free_limit":None,
          "over_max_size":None,"invalid_object":None,
          "bad_host_node":None,"ipfs_pin_hash":response["value"]["cid"],
          "limit":None,"offset":None,"metadata[name]":None,
          "metadatakeyvalues":"keyvalues"
         }

pinata.get_pinned_jobs(pinata_creds,params)

({'count': 1,
  'rows': [{'id': 'f9878627-7086-4018-864c-6bb26e5b553d',
    'ipfs_pin_hash': 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rvwkpjfr6tsj3ye',
    'date_queued': '2022-05-17T18:04:03.795Z',
    'name': 'ens_airdrop.json',
    'status': 'prechecking',
    'keyvalues': {},
    'host_nodes': None,
    'pin_policy': {'regions': [{'id': 'NYC1', 'desiredReplicationCount': 1}],
     'version': 1}}]},
 200)

In [9]:
"""
Query Parameters = params

hashContains: (string) - Filter on alphanumeric characters inside of pin hashes. Hashes which do not include the characters passed in will not be returned.

pinStart: (must be in ISO_8601 format) - Exclude pin records that were pinned before the passed in "pinStart" datetime.

pinEnd: (must be in ISO_8601 format) - Exclude pin records that were pinned after the passed in "pinEnd" datetime.

unpinStart: (must be in ISO_8601 format) - Exclude pin records that were unpinned before the passed in "unpinStart" datetime.

unpinEnd: (must be in ISO_8601 format) - Exclude pin records that were unpinned after the passed in "unpinEnd" datetime.

pinSizeMin: (integer) - The minimum byte size that pin record you're looking for can have

pinSizeMax: (integer) - The maximum byte size that pin record you're looking for can have

status: (string) -
    * Pass in "all" for both pinned and unpinned records
    * Pass in "pinned" for just pinned records (hashes that are currently pinned)
    * Pass in "unpinned" for just unpinned records (previous hashes that are no longer being pinned on pinata)

pageLimit: (integer) - This sets the amount of records that will be returned per API response. (Max 1000)

pageOffset: (integer) - This tells the API how far to offset the record responses. For example, 
if there's 30 records that match your query, and you passed in a pageLimit of 10, providing a pageOffset of 10 would return records 11-20.
"""

#Get all pinned files to unpin and then pin in NFT Storage
#If pageLimit == None then defaults to most recent 10

params = {"hasContains":"b","pinStart":None,
          "pinEnd":None,"unpinStart":None,
          "unpinEnd":None,"pinSizeMin":None,
          "pinSizeMax":None,"status":None,
          "pageLimit":100,"pageOffset":None,
         }

pf,status = pinata.get_pinned_files(pinata_creds,params)
df_pinata = pd.DataFrame(pf["rows"])

df_pinata.drop_duplicates(["ipfs_pin_hash"])

,id,ipfs_pin_hash,size,user_id,date_pinned,date_unpinned,metadata,regions
0,aeeedbc4-b9b4-4db8-bdf6-72026692bb3f,bafkreidqulmh3jp2oeea2fth7q3ilx3y456magfqhnj22...,210533,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-04T16:20:40.597Z,None,{'name': 'DefiSquad_Compound_StableCoin_Borrow...,"[{'regionId': 'NYC1', 'currentReplicationCount..."
1,60e6a6ea-87bb-4d46-8118-28cdd2fc719f,bafybeibpuxulnxkxyamy67y6ybyc4mnyjoaddjuxuuuhl...,264554,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-02T16:09:38.446Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
2,2fc660d2-00b8-45a2-9f53-525ad5de4218,bafkreibddlkj4ze3sogwp7cahtxvke7yggteayjkaaavd...,1698,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-02T16:09:17.391Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
3,2df4c71a-8666-4710-8306-0232c38102df,bafkreie2qucmecsyc3q7hhrccsif3lzsdu7zub3fy3zsp...,35270,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-02T16:07:53.923Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
4,506ae9a4-dc1d-4a62-b6e0-290fef1bfae9,bafkreiaxeimcjute2c6ecay6tulotm7nehgnhaolp4yca...,1698,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-02T16:07:35.393Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
5,653c4f52-8a86-443d-bf13-dee9eb7e1e63,bafkreicxtodg6u6qs23g6lbzyxkrvhua3bwu2ngz45ibt...,42739,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-01T16:13:23.891Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
6,ed61bba0-bef6-4afd-a225-162ac9360294,bafybeict3mhosmy3kdgoslpauxbadciaelmcdkrnopwaf...,285760,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-01T16:13:02.896Z,None,"{'name': '_', 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
7,4799e80f-ff91-4992-8854-27ed0d2db1ac,bafkreia4m7wo5kbjsy6qe2jlnouayh3ayzb32lvyrckrl...,30534,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-01T16:12:24.800Z,None,"{'name': '_', 'keyvalues': {'library': 'Sklear...","[{'regionId': 'NYC1', 'currentReplicationCount..."
8,ac6bf429-d7e5-45e8-b2c9-30919b01a26b,bafkreigd25ljwfzdqfjpe25zinufv66fqjxr4jy3hm3if...,63614,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-01T16:12:04.084Z,None,"{'name': '_', 'keyvalues': {'library': 'Sklear...","[{'regionId': 'NYC1', 'currentReplicationCount..."
9,3bb6cefd-d920-4e0a-b533-72af30cd6dd5,bafkreiayatfvb4pypssigt3x3lwpw5d3vebq77mylnbnu...,43548,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-05-01T16:11:31.520Z,None,"{'name': '_', 'keyvalues': {'library': 'Sklear...","[{'regionId': 'NYC1', 'currentReplicationCount..."


In [10]:
df_pinata.query('ipfs_pin_hash == "QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e"')["metadata"]

Series([], Name: metadata, dtype: object)

In [12]:
#Make a CID dictionary to pin these files to NFT Storage

df_pinata["filename"]=df_pinata["metadata"].apply(lambda x: x["name"])
file_dict = df_pinata[["filename","ipfs_pin_hash"]].drop_duplicates(subset="filename").set_index("filename")

file_dict

,ipfs_pin_hash
filename,
DefiSquad_Compound_StableCoin_BorrowingStrategy.jpg,bafkreidqulmh3jp2oeea2fth7q3ilx3y456magfqhnj22...
_,bafybeibpuxulnxkxyamy67y6ybyc4mnyjoaddjuxuuuhl...
RandomForestClassifier_5_7.pkl,QmUWHKNi9WLQPbD9sHFjYr5BrtyQ2ZVSLMrrGD6tmCKbZQ
AdaBoostClassifier_5_7.pkl,QmPyjZ1dpz2bbHxxC8F959QGQ3qPPw5PPAeJWZ1LJ6NCtw
RandomForestClassifierstratified_5_7.pkl,Qmbs43cCcWUYtXnq3EAdYk2Sj2oj26KJtXtYNz9cbD7W3U
GradientBoostingClassifierstratified_5_7.pkl,QmQF2ntZSTS5gZ94HqbXopdwzbdZBRVEW6sxgKYsDu9XGT
None,bafkreib6uzvfbv43ioa5eflfozyaccnv3qfmkc5vhwccw...


In [13]:
#NFT Storage Files

response = nft_storage.get_all_files(nft_storage_creds)

nft_storage_files = pd.DataFrame(response.json()["value"])

nft_storage_files

,cid,created,type,scope,files,size,name,pin,deals
0,bafkreidqulmh3jp2oeea2fth7q3ilx3y456magfqhnj22...,2022-05-04T16:20:32.234+00:00,form-data,audiocentive,[],210533,Upload at 2022-05-06T16:04:57.297Z,{'cid': 'bafkreidqulmh3jp2oeea2fth7q3ilx3y456m...,"[{'status': 'active', 'lastChanged': '2022-05-..."
1,bafybeibpuxulnxkxyamy67y6ybyc4mnyjoaddjuxuuuhl...,2022-05-02T16:09:27.062+00:00,form-data,audiocentive,[],264554,Upload at 2022-05-02T16:09:27.062Z,{'cid': 'bafybeibpuxulnxkxyamy67y6ybyc4mnyjoad...,"[{'status': 'active', 'lastChanged': '2022-05-..."
2,bafkreibddlkj4ze3sogwp7cahtxvke7yggteayjkaaavd...,2022-05-02T16:09:04.33+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-02T16:09:04.330Z,{'cid': 'bafkreibddlkj4ze3sogwp7cahtxvke7yggte...,"[{'status': 'active', 'lastChanged': '2022-05-..."
3,bafkreie2qucmecsyc3q7hhrccsif3lzsdu7zub3fy3zsp...,2022-05-02T16:07:46.353+00:00,form-data,audiocentive,[],35270,Upload at 2022-05-02T16:07:46.353Z,{'cid': 'bafkreie2qucmecsyc3q7hhrccsif3lzsdu7z...,"[{'status': 'active', 'lastChanged': '2022-05-..."
4,bafkreiaxeimcjute2c6ecay6tulotm7nehgnhaolp4yca...,2022-05-02T16:07:27.46+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-02T16:07:27.460Z,{'cid': 'bafkreiaxeimcjute2c6ecay6tulotm7nehgn...,"[{'status': 'active', 'lastChanged': '2022-05-..."
5,bafkreicxtodg6u6qs23g6lbzyxkrvhua3bwu2ngz45ibt...,2022-05-01T14:48:58.689+00:00,form-data,audiocentive,[],42739,Upload at 2022-05-01T16:13:14.268Z,{'cid': 'bafkreicxtodg6u6qs23g6lbzyxkrvhua3bwu...,"[{'status': 'active', 'lastChanged': '2022-05-..."
6,bafybeict3mhosmy3kdgoslpauxbadciaelmcdkrnopwaf...,2022-05-01T14:47:15.352+00:00,form-data,audiocentive,[],285760,Upload at 2022-05-01T16:12:46.559Z,{'cid': 'bafybeict3mhosmy3kdgoslpauxbadciaelmc...,"[{'status': 'active', 'lastChanged': '2022-05-..."
7,bafybeib6bki5chp33q2un4rcqxbl5lozwnjgzrkmzgmy7...,2022-05-01T05:17:49.636+00:00,form-data,audiocentive,[],268545,Upload at 2022-05-01T05:17:49.636Z,{'cid': 'bafybeib6bki5chp33q2un4rcqxbl5lozwnjg...,"[{'status': 'active', 'lastChanged': '2022-05-..."
8,bafkreib6tjtu4yrzrxazxd2sbyi6hro2h2ho3ntvgl3q6...,2022-05-01T05:17:29.324+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-01T05:17:29.324Z,{'cid': 'bafkreib6tjtu4yrzrxazxd2sbyi6hro2h2ho...,"[{'status': 'active', 'lastChanged': '2022-05-..."
9,bafkreieukwyd35jpv5pvfbb5bv2qowzzj3d5qakcuzld4...,2022-05-01T05:16:25.805+00:00,form-data,audiocentive,[],35270,Upload at 2022-05-01T05:16:25.805Z,{'cid': 'bafkreieukwyd35jpv5pvfbb5bv2qowzzj3d5...,"[{'status': 'active', 'lastChanged': '2022-05-..."


# Compare Pinata Files to NFT Storage to upload the outstanding files

In [14]:
pinata_pinned = file_dict["ipfs_pin_hash"].to_list()

nft_storage_pinned = nft_storage_files["cid"].to_list()

outstanding_hashs = set(nft_storage_pinned).difference(pinata_pinned)
outstanding_hashs

{'bafkreiaxeimcjute2c6ecay6tulotm7nehgnhaolp4ycayyvnkxly6jy3u',
 'bafkreib6tjtu4yrzrxazxd2sbyi6hro2h2ho3ntvgl3q6lsmwbn3b775hy',
 'bafkreibddlkj4ze3sogwp7cahtxvke7yggteayjkaaavdcvotx3wdsexgy',
 'bafkreicxtodg6u6qs23g6lbzyxkrvhua3bwu2ngz45ibtubvosby3a2uua',
 'bafkreie2qucmecsyc3q7hhrccsif3lzsdu7zub3fy3zspzggyrfjizjbeu',
 'bafkreieukwyd35jpv5pvfbb5bv2qowzzj3d5qakcuzld46zofvgxpzkfza',
 'bafybeib6bki5chp33q2un4rcqxbl5lozwnjgzrkmzgmy7jmu2fyqygbaye',
 'bafybeict3mhosmy3kdgoslpauxbadciaelmcdkrnopwaf6osszuzgdvbwq'}

In [15]:
#Pin oustanding hashes to pinata
for cid in outstanding_hashs:
    
    pinata.pin(pinata_creds,cid)

In [16]:
#Pin oustanding hashes to pinata

for cid in outstanding_hashs:
    pinata.unpin(pinata_creds,cid)
    nft_storage.unpin(nft_storage_creds,cid)

# As you can see all the CIDs with a Q were removed. Q is V1 and b is V2

In [17]:
response = nft_storage.get_all_files(nft_storage_creds)

nft_storage_files = pd.DataFrame(response.json()["value"])

nft_storage_files

,cid,created,type,scope,files,size,name,pin,deals
0,bafkreidqulmh3jp2oeea2fth7q3ilx3y456magfqhnj22...,2022-05-04T16:20:32.234+00:00,form-data,audiocentive,[],210533,Upload at 2022-05-06T16:04:57.297Z,{'cid': 'bafkreidqulmh3jp2oeea2fth7q3ilx3y456m...,"[{'status': 'active', 'lastChanged': '2022-05-..."
1,bafybeibpuxulnxkxyamy67y6ybyc4mnyjoaddjuxuuuhl...,2022-05-02T16:09:27.062+00:00,form-data,audiocentive,[],264554,Upload at 2022-05-02T16:09:27.062Z,{'cid': 'bafybeibpuxulnxkxyamy67y6ybyc4mnyjoad...,"[{'status': 'active', 'lastChanged': '2022-05-..."
2,bafkreihwhpx44sxz3qmaghrwmnhdbnjc54jqdnh3xjlck...,2022-05-01T05:16:08.837+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-01T05:16:08.837Z,{'cid': 'bafkreihwhpx44sxz3qmaghrwmnhdbnjc54jq...,"[{'status': 'active', 'lastChanged': '2022-05-..."
3,bafkreihmgzwftdylufh7sjfcvjydmbud22x6o5ezq5g74...,2022-05-01T05:05:51.314+00:00,form-data,audiocentive,[],257879,Upload at 2022-05-01T05:05:51.314Z,{'cid': 'bafkreihmgzwftdylufh7sjfcvjydmbud22x6...,"[{'status': 'active', 'lastChanged': '2022-05-..."
4,bafkreibaxy52fmi3iggbcks7bd6soedzbm7ljjy7wmqym...,2022-05-01T05:05:31.413+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-01T05:05:31.413Z,{'cid': 'bafkreibaxy52fmi3iggbcks7bd6soedzbm7l...,"[{'status': 'active', 'lastChanged': '2022-05-..."
5,bafkreibhkbtsouxafk6zklajriu4u5furd4dy6fhfrtho...,2022-05-01T05:04:19.429+00:00,form-data,audiocentive,[],35270,Upload at 2022-05-01T05:04:19.429Z,{'cid': 'bafkreibhkbtsouxafk6zklajriu4u5furd4d...,"[{'status': 'active', 'lastChanged': '2022-05-..."
6,bafkreiea3my7e6atl4jupmbeq6wf6uaglj6abzqnx7s5g...,2022-05-01T05:04:04.507+00:00,form-data,audiocentive,[],1698,Upload at 2022-05-01T05:04:04.507Z,{'cid': 'bafkreiea3my7e6atl4jupmbeq6wf6uaglj6a...,"[{'status': 'active', 'lastChanged': '2022-05-..."
7,bafybeig4lb73hzccn7srdpw5doucugp3bizee64cyfrt5...,2022-04-30T23:52:15.9+00:00,form-data,audiocentive,[],269357,Upload at 2022-04-30T23:52:15.900Z,{'cid': 'bafybeig4lb73hzccn7srdpw5doucugp3bize...,"[{'status': 'active', 'lastChanged': '2022-05-..."
8,bafkreicubfweo52qyyqzgzudjwz57msnuhon7xawxm2i5...,2022-04-30T23:51:57.864+00:00,form-data,audiocentive,[],1698,Upload at 2022-04-30T23:51:57.864Z,{'cid': 'bafkreicubfweo52qyyqzgzudjwz57msnuhon...,"[{'status': 'active', 'lastChanged': '2022-05-..."
9,bafkreia4rvpegipqja42ytgersqkxrsthzzhqmg4x4axq...,2022-04-30T23:50:50.929+00:00,form-data,audiocentive,[],35270,Upload at 2022-04-30T23:50:50.929Z,{'cid': 'bafkreia4rvpegipqja42ytgersqkxrsthzzh...,"[{'status': 'active', 'lastChanged': '2022-05-..."


In [18]:
from storage.ipfs import IPFS

ipfs = IPFS()

In [19]:
response,log = ipfs.get_file("QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",local_node=False)

Retrieved file hash QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e from https://dweb.link Response 200


In [20]:
read_file(response.content)

,Timestamp,Token,Borrowing Rate,Deposit Rate,Borrow Volume,Supply Volume
0,1609471800,DAI,0.073195,0.050982,1.069964e+09,6.196481e+10
1,1609471800,USDC,0.087046,0.066993,7.285430e+08,4.063042e+10
2,1609471800,USDT,0.099588,0.077548,6.430536e+07,3.696225e+09
3,1609471800,ETH,0.022952,0.000489,3.055365e+04,5.663257e+07
4,1609473600,DAI,0.073101,0.050912,1.069961e+09,6.197050e+10
...,...,...,...,...,...,...
66789,1639539000,ETH,0.027215,0.000761,5.316478e+04,7.481990e+07
66790,1639540800,ETH,0.027214,0.000761,5.316480e+04,7.482188e+07
66791,1639542600,ETH,0.027214,0.000761,5.316488e+04,7.482482e+07
66792,1639544400,ETH,0.027226,0.000764,5.316501e+04,7.461519e+07


In [21]:
data = {"ipfsPinHash": "QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",
        "name": "compound.csv",
        "keyvalues": {
            "DAO":"DeepFi",
            "Data":"Compound",
            "Task":"[BorrowRate,DepositRate]"
        }
         }

pinata.edit_hash(pinata_creds,"QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e","compound.csv",pinataMetaData=data)

200

In [25]:
data = {"ipfsPinHash": "QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",
        "name": "compound.csv",
        "keyvalues": {
            "DAO":"DeepFi",
            "Data":"Compound",
            "Task":None
        }
       }

pinata.upload_file(pinata_creds,"dataset/compound.csv","compound.csv",pinataMetadata=data)



({'IpfsHash': 'QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e',
  'PinSize': 5985301,
  'Timestamp': '2022-03-29T17:28:48.909Z',
  'isDuplicate': True},
 200)